In [77]:
#Packages
import math as ma
from math import sin
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
import openturns as ot
from openturns.viewer import View


In [78]:
data = pd.read_csv('data.txt', sep='\t')


In [79]:
#Define the parameter
#param(mean,stdv) Su, gamm excluded
# Su=[[mean],[std]], same for gamma
Su=[[],[]]
gamma=[[],[]]

for i in range(1,8):
    Su[0].append(data[5:].mean()["zone"+str(i)])
    Su[1].append(data[5:].std()["zone"+str(i)])
    gamma[0].append(data[:5].mean()["zone"+str(i)])
    gamma[1].append(data[:5].std()["zone"+str(i)])
e=[1,0.1]
a=[ma.radians(60),ma.radians(9)]
R=[8,1.6]
eta=[0.1,0.01]
q=[10,2]
Su={
    'mean' : Su[0],
    'std' : Su[1],
}

gamma={
    'mean' : gamma[0],
    'std' : gamma[1],
}

pd.DataFrame(Su, index = ['zone1','zone2','zone3','zone4','zone5','zone6','zone7'])


#get mean : print(Su['zone1'][0])


,mean,std
zone1,46.857143,8.468764
zone2,56.239161,7.557970
zone3,44.597203,9.732100
zone4,51.547153,5.190861
zone5,37.073526,6.058854
zone6,49.830569,9.153118
zone7,54.009690,11.091138


In [80]:
pd.DataFrame(gamma, index = ['zone1','zone2','zone3','zone4','zone5','zone6','zone7'])


,mean,std
zone1,18.40,1.019804
zone2,18.34,2.039117
zone3,17.30,2.732215
zone4,18.78,1.976613
zone5,18.72,1.395349
zone6,19.64,2.169793
zone7,16.88,2.591718


In [81]:
dist_e=ot.Normal(1,0.1)
dist_e.setDescription('e')

dist_a=ot.Normal(ma.radians(60),ma.radians(9))
dist_a.setDescription('a')

dist_R=ot.Normal(8,1.6)
dist_R.setDescription('R')

dist_q=ot.Normal(10,2)
dist_q.setDescription('q')

dist_eta=ot.Normal(0.1,0.01)
dist_eta.setDescription('n')

dist_Su=[]
dist_gamma=[]
for i in range (7):
    dist_Su.append(ot.Normal(Su['mean'][i],Su['std'][i]))
    dist_Su[i].setDescription('S')
    dist_gamma.append(ot.Normal(gamma['mean'][i],gamma['std'][i]))
    dist_gamma[i].setDescription('g')



In [82]:
marginals=[]
distributions=[]
for i in range (7):
    marginals.append([dist_e,dist_a, dist_R,dist_q,dist_eta,dist_Su[i],dist_gamma[i]])
    distributions.append(ot.ComposedDistribution(marginals[i]))
    distributions[i].setDescription(['e','a','R','q','n','S','g'])

model = ot.SymbolicFunction(['e','a','R','q','n','S','g'], ['e*S*a/(0.25*g*R*(a-sin(a))*sin(a)+n*q*sin(a))'])


In [83]:
vect=[]
G=[]
event=[]
for i in range (7):
    vect.append(ot.RandomVector(distributions[i]))
    G.append(ot.CompositeRandomVector(model, vect[i]))
    event.append(ot.ThresholdEvent(G[i], ot.Greater(), 1))
    event[i].setName("performance function g")


In [89]:
optimAlgo = ot.Cobyla()
optimAlgo.setMaximumEvaluationNumber(50000)
#optimAlgo.setMaximumAbsoluteError(0.001)
#optimAlgo.setMaximumRelativeError(0.001)
#optimAlgo.setMaximumResidualError(0.001)
#optimAlgo.setMaximumConstraintError(0.001)
algo=[]
results=[]
pf_Form=[]
for i in range (7):
    algo.append(ot.FORM(optimAlgo, event[i], distributions[i].getMean()))
    algo[i].run()
    results.append(algo[i].getResult())
    pf_Form.append(results[i].getEventProbability())
    print('Failure probability zone', str(i+1), ':' ,1-pf_Form[i])



Failure probability zone 1 : 3.3487571059476196e-06
Failure probability zone 2 : 2.1891336521306926e-09
Failure probability zone 3 : 7.231280111108518e-05
Failure probability zone 4 : 2.268807364202985e-10
Failure probability zone 5 : 5.6808530012331104e-06
Failure probability zone 6 : 5.026415411779617e-06
Failure probability zone 7 : 1.2850314349610592e-05


In [92]:
optimAlgo = ot.Cobyla()
#optimAlgo.setMaximumEvaluationNumber(50000)
optimAlgo.setMaximumAbsoluteError(0.01)
#optimAlgo.setMaximumRelativeError(0.001)
#optimAlgo.setMaximumResidualError(0.001)
#optimAlgo.setMaximumConstraintError(0.001)
algo=[]
results=[]
pf_Form=[]
for i in range (7):
    algo.append(ot.FORM(optimAlgo, event[i], distributions[i].getMean()))
    algo[i].run()
    results.append(algo[i].getResult())
    pf_Form.append(results[i].getEventProbability())
    print('Failure probability zone', str(i+1), ':' ,1-pf_Form[i])


Failure probability zone 1 : 3.348641976930189e-06
Failure probability zone 2 : 2.1889894341597937e-09
Failure probability zone 3 : 7.230836408422814e-05
Failure probability zone 4 : 2.2685564537994196e-10
Failure probability zone 5 : 5.680392882734253e-06
Failure probability zone 6 : 5.0258264511215955e-06
Failure probability zone 7 : 1.2849330184416807e-05
